In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda() 
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 1000
    xa1 = 2*torch.rand(Nb, dimension) - 1.0
    xa1[:, 0] = -torch.ones(Nb)
    
    xa2 = 2*torch.rand(Nb, dimension) - 1.0
    xa2[:, 1:3] = xa1[:, 1:3]
    xa2[:, 0] = torch.ones(Nb)
    
    xb1 = 2*torch.rand(Nb, dimension) - 1.0
    xb1[:, 1] = -torch.ones(Nb)
    
    xb2 = 2*torch.rand(Nb, dimension) - 1.0
    xb2[:, 0] = xb1[:, 0] 
    xb2[:, 2:3] = xb1[:, 2:3]
    xb2[:, 1] = torch.ones(Nb)
    
    xc1 = 2*torch.rand(Nb, dimension) - 1.0
    xc1[:, 2] = -torch.ones(Nb)
    
    xc2 = 2*torch.rand(Nb, dimension) - 1.0
    xc2[:, 0:1] = xc1[:, 0:1]
    xc2[:, 3] = xc1[:, 3]
    xc2[:, 2] = torch.ones(Nb)
    
    xd1 = 2*torch.rand(Nb, dimension) - 1.0
    xd1[:, 3] = -torch.ones(Nb)
    
    xd2 = 2*torch.rand(Nb, dimension) - 1.0
    xd2[:, 0:2] = xd1[:, 0:2]  
    xd2[:, 3] = torch.ones(Nb)
    
    xa1 = xa1.cuda()
    xa2 = xa2.cuda()
    xb1 = xb1.cuda()
    xb2 = xb2.cuda()
    xc1 = xc1.cuda()
    xc2 = xc2.cuda()
    xd1 = xd1.cuda()
    xd2 = xd2.cuda()
    
    pxa = ((model(xa1) - model(xa2))**2).cuda() 
    pxb = ((model(xb1) - model(xb2))**2).cuda()  
    pxc = ((model(xc1) - model(xc2))**2).cuda()  
    pxd = ((model(xd1) - model(xd2))**2).cuda()  
    part_2 = torch.sum((pxa + pxb + pxc + pxd)[:,0])/2/xa1.size()[0]/4
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx4[:, 3] = torch.ones(xb1.size()[0])
    
    dxa = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 - (model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2)**2).cuda()  
    dxb = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 - (model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2)**2).cuda()  
    dxc = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 - (model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2)**2).cuda()  
    dxd = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 - (model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2)**2).cuda()  
    part_3 = torch.sum((dxa + dxb + dxc + dxd)[:,0])/2/xb1.size()[0]/4
    
    lambda1 = 1.0
    lambda2 = 0.5
    return part_1 + lambda1 * part_2 + lambda2 * part_3

In [10]:
Data_size = 2000
def Gendata():
    x = 2*torch.rand(Data_size, dimension) - 1.0
    return x.cuda()  

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_4D_Periodic.npy", error_save)
np.save("DGM_relative_error_4D_Periodic.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(751.7804, device='cuda:0')
current relative error is:  tensor(1.0482, device='cuda:0')
current epoch is:  50
current loss is:  tensor(583.1986, device='cuda:0')
current relative error is:  tensor(0.9644, device='cuda:0')
current epoch is:  100
current loss is:  tensor(441.6673, device='cuda:0')
current relative error is:  tensor(0.9834, device='cuda:0')
current epoch is:  150
current loss is:  tensor(244.7868, device='cuda:0')
current relative error is:  tensor(1.1316, device='cuda:0')
current epoch is:  200
current loss is:  tensor(118.7783, device='cuda:0')
current relative error is:  tensor(1.3975, device='cuda:0')
current epoch is:  250
current loss is:  tensor(75.9927, device='cuda:0')
current relative error is:  tensor(1.3651, device='cuda:0')
current epoch is:  300
current loss is:  tensor(57.9807, device='cuda:0')
current relative error is:  tensor(1.3387, device='cuda:0')
current epoch is:  350
current loss is:  tensor(51.1667, dev

current epoch is:  3100
current loss is:  tensor(4.5104, device='cuda:0')
current relative error is:  tensor(0.4307, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(4.6244, device='cuda:0')
current relative error is:  tensor(0.4134, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(4.3771, device='cuda:0')
current relative error is:  tensor(0.4196, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(4.3154, device='cuda:0')
current relative error is:  tensor(0.3914, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(4.1498, device='cuda:0')
current relative error is:  tensor(0.3934, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(3.9847, device='cuda:0')
current relative error is:  tensor(0.3968, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(4.0071, device='cuda:0')
current relative error is:  tensor(0.4008, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(4.0628, devic

current epoch is:  6200
current loss is:  tensor(1.6728, device='cuda:0')
current relative error is:  tensor(0.1879, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(1.6709, device='cuda:0')
current relative error is:  tensor(0.1768, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(1.5651, device='cuda:0')
current relative error is:  tensor(0.1684, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(1.5956, device='cuda:0')
current relative error is:  tensor(0.1678, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(1.6299, device='cuda:0')
current relative error is:  tensor(0.1663, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(1.6508, device='cuda:0')
current relative error is:  tensor(0.1614, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(1.6281, device='cuda:0')
current relative error is:  tensor(0.1735, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(1.4710, devic

current epoch is:  9300
current loss is:  tensor(1.0275, device='cuda:0')
current relative error is:  tensor(0.0835, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(1.0008, device='cuda:0')
current relative error is:  tensor(0.0802, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(1.0068, device='cuda:0')
current relative error is:  tensor(0.0790, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(1.0630, device='cuda:0')
current relative error is:  tensor(0.0808, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(1.0006, device='cuda:0')
current relative error is:  tensor(0.0763, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(1.0089, device='cuda:0')
current relative error is:  tensor(0.0764, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(1.0177, device='cuda:0')
current relative error is:  tensor(0.0752, device='cuda:0')
current epoch is:  9650
current loss is:  tensor(1.0087, devic

current epoch is:  12350
current loss is:  tensor(0.9037, device='cuda:0')
current relative error is:  tensor(0.0555, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(0.8633, device='cuda:0')
current relative error is:  tensor(0.0576, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(0.8687, device='cuda:0')
current relative error is:  tensor(0.0521, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.9933, device='cuda:0')
current relative error is:  tensor(0.0597, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.8497, device='cuda:0')
current relative error is:  tensor(0.0584, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.8869, device='cuda:0')
current relative error is:  tensor(0.0572, device='cuda:0')
current epoch is:  12650
current loss is:  tensor(0.8617, device='cuda:0')
current relative error is:  tensor(0.0565, device='cuda:0')
current epoch is:  12700
current loss is:  tensor(0.819

current epoch is:  15400
current loss is:  tensor(0.7766, device='cuda:0')
current relative error is:  tensor(0.0479, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.7876, device='cuda:0')
current relative error is:  tensor(0.0521, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.7822, device='cuda:0')
current relative error is:  tensor(0.0486, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.8040, device='cuda:0')
current relative error is:  tensor(0.0525, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.7735, device='cuda:0')
current relative error is:  tensor(0.0454, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.7754, device='cuda:0')
current relative error is:  tensor(0.0474, device='cuda:0')
current epoch is:  15700
current loss is:  tensor(0.7847, device='cuda:0')
current relative error is:  tensor(0.0519, device='cuda:0')
current epoch is:  15750
current loss is:  tensor(0.813

current epoch is:  18450
current loss is:  tensor(0.7265, device='cuda:0')
current relative error is:  tensor(0.0466, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(0.7300, device='cuda:0')
current relative error is:  tensor(0.0442, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(0.7195, device='cuda:0')
current relative error is:  tensor(0.0441, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(0.7217, device='cuda:0')
current relative error is:  tensor(0.0426, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(0.7341, device='cuda:0')
current relative error is:  tensor(0.0425, device='cuda:0')
current epoch is:  18700
current loss is:  tensor(0.7309, device='cuda:0')
current relative error is:  tensor(0.0442, device='cuda:0')
current epoch is:  18750
current loss is:  tensor(0.7435, device='cuda:0')
current relative error is:  tensor(0.0463, device='cuda:0')
current epoch is:  18800
current loss is:  tensor(0.706